In [2]:
import cv2
import numpy as np
from matplotlib  import pyplot as plt
from helpers import helpers_formulate_lanes as formulate_lanes

In [3]:
def mainfunc(greyimg,low_th,high_th,minlinelin,maxlinegap,orgimg):
    aftcanny=cannyimg(greyimg,low_th,high_th)
    aftmask=maskROI(aftcanny)
    afthoughline=hough_lines(aftcanny,minlinelin,maxlinegap)
    aftform=formulate_lanes(afthoughline,aftmask)
    lanes_img = draw_lines(aftform, orgimg)
    return cv2.addWeighted(orgimg, 0.9, lanes_img, 1, 0)
def draw_lines(lines, masked_edges):
  color = [243, 105, 14]
  thickness = 12
  lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
  for line in lines:
    for x1,y1,x2,y2 in line:
      cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
  return(lines_image)
def hough_lines(img,min_line_len,max_line_gap):
  RHO = 1                 # try: 1 - 4 (0.5 increments)  
  THETA = np.pi/180       # Usually this is Ok
  MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
  #MIN_LINE_LEN = 5 
  #MAX_LINE_GAP = 50 
  lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
  return lines
def cannyimg(img,low_th,high_th):
  return cv2.Canny(cv2.GaussianBlur(img, (7,7), 0), low_th, high_th)
def maskROI(img):
  mask = np.zeros_like(img)
  if len(img.shape) > 2:
    channel_count = img.shape[2]
    ignore_mask_color = (255,) * channel_count
  else:
    ignore_mask_color = 255
  imshape = img.shape        
  vertices = np.array([
    [
      ((1/6*imshape[1]), imshape[0]),
      ((5/12*imshape[1]), (3/5*imshape[0])),
      ((7/12*imshape[1]), (3/5*imshape[0])),
      ((9/10*imshape[1]), imshape[0])
    ]], dtype=np.int32)
  cv2.fillPoly(mask, vertices, ignore_mask_color)
  masked_edges = cv2.bitwise_and(img, mask)
  return(masked_edges)


In [4]:
img1=cv2.imread("nature.jpg")
img2=cv2.imread("Task_in.jpg")
print(img1.shape,img2.shape)

(331, 600, 3) (320, 480, 3)


In [7]:
cv2.namedWindow("image")
def dum(x):
    pass
cv2.createTrackbar("imgno","image",0,1,dum)
cv2.createTrackbar("cannyorblur","image",0,1,dum)
while(True):
    imgno=cv2.getTrackbarPos("imgno","image")
    cannyorblur=cv2.getTrackbarPos("cannyorblur","image")
    img=np.array([])
    if imgno==0:
        img=np.copy(img1)
    else:
        img=np.copy(img2)
    if cannyorblur==0:
        out=cannyimg(img,52,123)
        #mainfunc(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),52,123,40,0,img)
    else:
        out=cv2.blur(img,(9,9))
    cv2.imshow("image",out)
    key=cv2.waitKey(1)
    if(key==ord('q')):
        break
cv2.destroyAllWindows()    

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2581: error: (-27:Null pointer) NULL window: 'image' in function 'cvGetTrackbarPos'
